# Guía 4

Nombre estudiante: Pablo Gómez Paredes  
Nombre profesor: David Medina Ortiz  
Asignatura: Matemáticas para Ciencias de la Computación  
Fecha entrega: 28/11/2025

## 1.1) Introducción: Colección, preparación y justificación del dataset escogido

En la presente guía se analizará un dataset referente a los datos históricos del tiempo en Punta Arenas, abarcando el período desde el 1 de septiembre hasta el 31 de octubre del presente año.  Los descriptores incluidos corresponden principalmente a temperatura, velocidad del viento y presión atmosférica.

La etiqueta _(label)_ que se analizará en función de estos descriptores será un valor binario (0 o 1) construido específicamente para este análisis, indicando si en un día determinado hubo precipitación (1) o no la hubo (0).  Esta decisión permitirá simplificar el análisis, y centrar la exploración estadística en la ocurrencia del fenómeno y en qué factores pueden hacer más probable que ocurra ese fenómeno, más que en su intensidad.

Este dataset se escogió principalmente porque permite trabajar con datos reales, recientes y que además son más de mi interés analizar.   Y por otra parte, también porque el clima de Punta Arenas presenta condiciones particularmente dinámicas —con fuertes variaciones de viento, cambios bruscos de temperatura y frecuencia de precipitaciones— lo que genera un conjunto de datos estadísticamente interesante para estudiar asociaciones y patrones.

Para la obtención del dataset, hubo que instalar la librería _meteostat_ a través del comando pip install.  Esta librería facilita el acceso a registros metereológicos de estaciones oficiales y permite descargar datos fácilmente transformables para su posterior análisis.

In [48]:
# Inclusión de librerías necesarias
from meteostat import Daily, Point
import pandas as pd
from datetime import datetime
from auxiliar_functions import *

In [49]:
# Coordenadas de Punta Arenas
punta_arenas = Point(-53.155877, -70.901687)

In [50]:
# Rango de fechas
start = datetime(2025, 9, 1)
end = datetime(2025, 10, 31)

In [58]:
# Descarga de los datos
data = Daily(punta_arenas, start, end)
df = data.fetch()

## 1.2) Descripción de metadatos y composición del dataset

En las siguientes celdas se hará el código que permitirá describir los metadatos y la composición del dataset, y también luego se explicará.

In [59]:
# Primeras filas del dataset
df.head(10)

,tavg,tmin,tmax,prcp,snow,wdir,wspd,wpgt,pres,tsun
time,,,,,,,,,,
2025-09-01,4.0,1.0,7.0,0.4,<NA>,<NA>,31.7,<NA>,1001.1,<NA>
2025-09-02,1.4,-1.0,4.0,0.0,<NA>,<NA>,33.2,<NA>,996.5,<NA>
2025-09-03,2.4,-1.0,5.0,0.0,<NA>,<NA>,33.8,<NA>,1014.6,<NA>
2025-09-04,5.9,3.0,9.0,0.0,<NA>,<NA>,27.1,<NA>,1022.4,<NA>
2025-09-05,7.4,5.0,10.0,0.4,<NA>,<NA>,36.2,<NA>,1016.2,<NA>
2025-09-06,6.8,5.0,10.0,0.0,<NA>,<NA>,33.8,<NA>,1011.9,<NA>
2025-09-07,6.3,4.0,10.0,0.0,<NA>,<NA>,35.6,<NA>,1006.1,<NA>
2025-09-08,5.4,2.0,9.0,0.4,<NA>,<NA>,19.5,<NA>,1010.7,<NA>
2025-09-09,5.2,4.0,6.0,3.6,<NA>,<NA>,29.0,<NA>,1003.5,<NA>


In [60]:
# Tamaño del dataset
df.shape

(61, 10)

In [61]:
# Tipos de metadatos
df.dtypes

tavg    Float64
tmin    Float64
tmax    Float64
prcp    Float64
snow    Float64
wdir    Float64
wspd    Float64
wpgt    Float64
pres    Float64
tsun    Float64
dtype: object

El dataset en su forma original contiene 61 ejemplos y 10 descriptores, todos del tipo numérico, los cuales son:

* tavg: La temperatura media de un día (°C).
* tmin: La temperatura mínima alcanzada durante un día (°C).
* tmax: La temperatura máxima alcanzada durante un día (°C).
* prcp: La precipitación durante un día (mm)._
* snow: La precipitación en forma de nieve durante un día (mm).
* wdir: Dirección del viento (°).
* wspd: Rapidez media del viento durante un día (km/h).
* wpgt: Rapidez máxima de las rachas de viento (km/h).
* pres: Presión atmosférica media a nivel del mar (hPa).
* tsun: Horas de sol diarias (h).

Nota: Incluso podrían ser 11 descriptores si consideramos también a la fecha como un descriptor más.

El dataset no incluye originalmente una etiqueta _(label)_ para realizar análisis.  Por este motivo, y dado que el objetivo de la guía es investigar asociaciones entre descriptores y un fenómeno de interés, se generará una nueva etiqueta binaria basada en la ocurrencia de precipitación.  Específicamente, la columna prcp será recodificada para producir una variable 0/1 que indique si en un día determinado hubo lluvia (1) o no la hubo (0).

Al observar los descriptores, también se aprecia a simple vista que algunos presentan una cantidad importante de valores faltantes. Para evaluar su impacto, se realizará un recuento formal de datos nulos por columna. Como criterio general — común en análisis exploratorio — si un descriptor presenta más de un 30% de valores faltantes, la columna será eliminada debido a su baja utilidad informativa. En caso de porcentajes menores, y si la variable resulta relevante para el análisis, sus valores faltantes se imputarán mediante la mediana, considerando que se trata de variables numéricas potencialmente afectadas por asimetría.

Cabe destacar que, al tratarse de datos meteorológicos históricos, sería posible completar los valores faltantes consultando registros de otras estaciones meteorológicas. Sin embargo, dicha reconstrucción iría más allá del propósito de esta guía, cuyo objetivo es practicar análisis exploratorio y tratamiento estadístico de datos tal como se presentan.

## 2.1) EDA y descripción de descriptores

In [62]:
# Se genera el recuento de nulos para cada uno de los descriptores
df_count_nulls = generate_df_count_nulls(df=df)
df_count_nulls

,count_nulls,pct_nulls
tavg,0,0.0
tmin,0,0.0
tmax,0,0.0
prcp,0,0.0
snow,61,100.0
wdir,61,100.0
wspd,0,0.0
wpgt,61,100.0
pres,0,0.0
tsun,61,100.0


In [ ]:
columns_to_delete =  []

# Se eliminan las columnas que tienen al menos un 30% de datos nulos, en caso contrario habría que hacer imputación por medio de la mediana
for row in df_count_nulls.index:
    if df_count_nulls["pct_nulls"][row] >= 30.0:
        columns_to_delete.append(row)

df = df.drop(columns=columns_to_delete)

df.head(10)  # Se muestran las primeras filas (registros) del dataset

,tavg,tmin,tmax,prcp,wspd,pres
time,,,,,,
2025-09-01,4.0,1.0,7.0,0.4,31.7,1001.1
2025-09-02,1.4,-1.0,4.0,0.0,33.2,996.5
2025-09-03,2.4,-1.0,5.0,0.0,33.8,1014.6
2025-09-04,5.9,3.0,9.0,0.0,27.1,1022.4
2025-09-05,7.4,5.0,10.0,0.4,36.2,1016.2
2025-09-06,6.8,5.0,10.0,0.0,33.8,1011.9
2025-09-07,6.3,4.0,10.0,0.0,35.6,1006.1
2025-09-08,5.4,2.0,9.0,0.4,19.5,1010.7
2025-09-09,5.2,4.0,6.0,3.6,29.0,1003.5


In [65]:
df = df.rename(columns={"prcp": "mm"})
df["prcp"] = df["mm"].apply(lambda x: 0 if x == 0.0 else 1)  # Aquí se genera el label (originalmente no venía)
df = df.drop(columns=["mm"])
df

,tavg,tmin,tmax,wspd,pres,prcp
time,,,,,,
2025-09-01,4.0,1.0,7.0,31.7,1001.1,1
2025-09-02,1.4,-1.0,4.0,33.2,996.5,0
2025-09-03,2.4,-1.0,5.0,33.8,1014.6,0
2025-09-04,5.9,3.0,9.0,27.1,1022.4,0
2025-09-05,7.4,5.0,10.0,36.2,1016.2,1
...,...,...,...,...,...,...
2025-10-27,6.5,-2.0,15.0,9.8,1024.2,0
2025-10-28,8.2,4.0,12.0,11.9,1011.8,0
2025-10-29,11.4,8.0,16.0,27.6,1006.7,0


In [66]:
# Verificamos que el dataset nuevo no tiene datos nulos
new_df_count_nulls = generate_df_count_nulls(df=df)
new_df_count_nulls

,count_nulls,pct_nulls
tavg,0,0.0
tmin,0,0.0
tmax,0,0.0
wspd,0,0.0
pres,0,0.0
prcp,0,0.0


In [46]:
df_descriptive = df.describe().T
df_descriptive = df_descriptive.drop(columns=["count"])
df_descriptive = df_descriptive.apply(pd.to_numeric, errors="coerce")
df_descriptive["mean"] = df_descriptive["mean"].round(decimals=2)
df_descriptive["std"] = df_descriptive["mean"].round(decimals=2)
df_descriptive


,mean,std,min,25%,50%,75%,max
tavg,6.13,6.13,1.4,4.8,6.4,7.2,11.4
tmin,2.77,2.77,-3.0,1.0,3.0,4.0,8.0
tmax,9.89,9.89,4.0,8.0,10.0,11.0,17.0
wspd,26.08,26.08,8.0,20.1,27.1,32.1,41.7
pres,1001.12,1001.12,973.0,993.5,999.3,1009.2,1024.8
prcp,0.39,0.39,0.0,0.0,0.0,1.0,1.0
